In [1]:
import requests
import re
import json
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup
import sys
sys.path.insert(0, '../data_prep')
from sqlite_utils import get_from_table

In [2]:
film_id = 'f_01U58'
film_title = get_from_table('FILM_TITLE', film_id, 'FILM_TITLE')
letterboxd_url = get_from_table('FILM_TITLE', film_id, 'LETTERBOXD_URL')
print('Film ID is "{}", film title is "{}", and url is {}'.format(film_id, film_title, letterboxd_url))

Film ID is "f_01U58", film title is "Salem's Lot", and url is https://boxd.it/1U58


In [3]:
r = requests.get(letterboxd_url)
redirected_url = r.url
if letterboxd_url != redirected_url:
    r = requests.get(redirected_url)
soup = BeautifulSoup(r.content, 'lxml')

<!DOCTYPE html>
<!--[if lt IE 7 ]> <html lang="en" class="ie6 lte9 lte8 lte7 lte6 no-js"> <![endif]--><!--[if IE 7 ]>    <html lang="en" class="ie7 lte9 lte8 lte7 no-js"> <![endif]--><!--[if IE 8 ]>    <html lang="en" class="ie8 lte9 lte8 no-js"> <![endif]--><!--[if IE 9 ]>    <html lang="en" class="ie9 lte9 no-js"> <![endif]--><!--[if (gt IE 9)|!(IE)]><!--><html class="no-mobile no-js" id="html" lang="en"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="width=1024" name="viewport"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="Ben Mears has returned to his hometown to write a book about the supposedly haunted Marsten House. When people around the Marsten House start dying mysteriously, Mears discovers that the owner of the mansion is actually a vampire who is turning them into an army of undead slaves." name="description"/>
<meta content="video.movie" property="og:type"/>
<meta content="https://letterboxd.com/film/salems-lot/" property=

In [4]:
tmdb_url = soup.find('a', {'data-track-action': 'TMDb'}).get('href')
tmdb_url

'https://www.themoviedb.org/tv/16118/'

In [18]:
imdb_url = soup.find('a', {'data-track-action': 'IMDb'}).get('href')
imdb_id = imdb_url.replace('http://www.imdb.com/title/', '').replace('/maindetails', '')
print(imdb_url, imdb_id)

http://www.imdb.com/title/tt0079844/maindetails tt0079844


'tt0079844'

In [12]:
pos = str(soup).find('imdb')
str(soup)[pos-50:pos+100]

'-event" data-track-action="IMDb" href="http://www.imdb.com/title/tt0079844/maindetails" target="_blank">IMDb</a>\n<a class="micro-button track-event" d'

In [ ]:
if tmdb_url[27:32] == 'movie':
    tmdb_id = tmdb_url.replace('https://www.themoviedb.org/movie/', '').replace('/', '')
    content_type = 'movie'
elif tmdb_url[27:29] == 'tv':
    tmdb_id = tmdb_url.replace('https://www.themoviedb.org/tv/', '').replace('/', '')
    content_type = 'tv'
else:
    return
tmdb_dict = {
    'FILM_ID': film_id,
    'TMDB_ID':tmdb_id
}

In [ ]:
if imdb_url[27:32] == 'movie':
    tmdb_id = tmdb_url.replace('https://www.themoviedb.org/movie/', '').replace('/', '')
    content_type = 'movie'
elif tmdb_url[27:29] == 'tv':
    tmdb_id = tmdb_url.replace('https://www.themoviedb.org/tv/', '').replace('/', '')
    content_type = 'tv'
else:
    return
tmdb_dict = {
    'FILM_ID': film_id,
    'TMDB_ID':tmdb_id
}

In [37]:
letterboxd_url = get_from_table('FILM_TITLE', film_id, 'LETTERBOXD_URL')
r = requests.get(letterboxd_url)
if r.status_code != 200:
    print('STATUS CODE NOT 200')
redirected_url = r.url
if letterboxd_url != redirected_url:
    r = requests.get(redirected_url)
soup = BeautifulSoup(r.content, 'lxml')
tmdb_url = soup.find('a', {'data-track-action': 'TMDb'}).get('href')

In [38]:
tmdb_url

'https://www.themoviedb.org/tv/79788/'

In [41]:
if tmdb_url[27:32] == 'movie':
    tmdb_id = tmdb_url.replace('https://www.themoviedb.org/movie/', '').replace('/', '')
    content_type = 'movie'
elif tmdb_url[27:29] == 'tv':
    tmdb_id = tmdb_url.replace('https://www.themoviedb.org/tv/', '').replace('/', '')
    content_type = 'tv'

In [42]:
print(content_type, tmdb_id)

tv 79788


In [ ]:
tmdb_id = tmdb_url.replace('https://www.themoviedb.org/movie/', '').replace('/', '')
tmdb_dict = {
    'FILM_ID': film_id,
    'TMDB_ID':tmdb_id
}
# insert_record_into_table(tmdb_dict, 'TMDB_ID')

In [5]:
r = requests.get(letterboxd_url)
r

<Response [200]>

In [6]:
redirected_url = r.url
if letterboxd_url != redirected_url:
    r = requests.get(redirected_url)

In [7]:
soup = BeautifulSoup(r.content, 'lxml')

In [8]:
tmdb_url = soup.find('a', {'data-track-action': 'TMDb'}).get('href')
tmdb_id = tmdb_url.replace('https://www.themoviedb.org/movie/', '').replace('/', '')

In [32]:
og_url = soup.find('meta', {'property': 'og:url'}).get('content')
film = og_url.split('/')[-2]
try:
    year = int(list(re.search(r'\((.*?)\)', soup.find('meta', {'property': 'og:title'}).get('content')).groups())[0])
except:
    year = int(datetime.now().strftime('%Y')) + 2
genre_list = [x.get('href').replace('/films/genre/', '').replace('/', '') for x in soup.findAll('a', {'class':'text-slug'}) if 'genre' in str(x.get('href'))]

rating_dict = json.loads(soup.find('script', {'type':"application/ld+json"}).string.split('\n')[2]).get('aggregateRating')
try:
    rating_mean = rating_dict.get('ratingValue')
except:
    rating_mean = np.nan
try:
    rating_count = rating_dict.get('ratingCount')
except:
    rating_count = np.nan

In [33]:
genre_list

[]

In [11]:
r = requests.get('https://letterboxd.com/film/{}/members/rated/.5-5/'.format(film))

In [12]:
soup = BeautifulSoup(r.content, 'lxml')

In [13]:
metrics_dict = {}
for i in ['members', 'fans', 'likes', 'reviews', 'lists']:
    href_str = '/film/{}/{}/'.format(film, i)
    try:
        metric_string = soup.find('a', {'class': 'tooltip', 'href':href_str}).get('title')
        metric = int(metric_string[:metric_string.find('\xa0')].replace(',', ''))
    except:
        metric = 0
    metrics_dict[i] = metric

In [14]:
metrics_dict

{'members': 1746806,
 'fans': 22197,
 'likes': 600578,
 'reviews': 91071,
 'lists': 188382}

In [15]:
r = requests.get('https://letterboxd.com/esi/film/{}/stats/'.format(film))
soup = BeautifulSoup(r.content, 'lxml')

In [16]:
try:
    top_ = int(soup.find('a', {'class': 'has-icon icon-top250 icon-16 tooltip'}).text)
except:
    top_ = None
letterboxd_info_dict = {
    'FILM_ID': film_id,
    'FILM_WATCH_COUNT': metrics_dict['members'],
    'FILM_FAN_COUNT': metrics_dict['fans'],
    'FILM_LIKES_COUNT': metrics_dict['likes'],
    'FILM_REVIEW_COUNT': metrics_dict['reviews'],
    'FILM_LIST_COUNT': metrics_dict['lists'],
    'FILM_TOP_250': top_,
    'FILM_RATING': rating_mean,
    'FILM_RATING_COUNT': rating_count,
}
letterboxd_info_dict

{'FILM_ID': 'f_02a1m',
 'FILM_WATCH_COUNT': 1746806,
 'FILM_FAN_COUNT': 22197,
 'FILM_LIKES_COUNT': 600578,
 'FILM_REVIEW_COUNT': 91071,
 'FILM_LIST_COUNT': 188382,
 'FILM_TOP_250': None,
 'FILM_RATING': 4.15,
 'FILM_RATING_COUNT': 993625}

In [34]:
film_genre_dict = {
        'FILM_ID': film_id,
        'FILM_GENRE':genre_list[0],
        'ALL_FILM_GENRES': '/'.join(genre_list)
    }
film_genre_dict

IndexError: list index out of range

In [18]:
from justwatch import JustWatch
# Update streaming info function:
just_watch = JustWatch(country='GB')
film_url_title = get_from_table('FILM_TITLE', film_id, 'FILM_URL_TITLE')
film_release_year = get_from_table('FILM_YEAR', film_id, 'FILM_YEAR')
results = just_watch.search_for_item(query=film_url_title, release_year_from=film_release_year-1, release_year_until=film_release_year+1)
first_result = results['items'][0]


In [19]:
first_result

{'jw_entity_id': 'ts20995',
 'id': 20995,
 'title': 'The Sopranos',
 'full_path': '/uk/tv-series/the-sopranos',
 'full_paths': {'SHOW_DETAIL_OVERVIEW': '/uk/tv-series/the-sopranos'},
 'poster': '/poster/9077158/{profile}',
 'poster_blur_hash': 'TnF#,f%Mbb_NxuoL-;j[WCwdWBWV',
 'object_type': 'show',
 'scoring': [{'provider_type': 'imdb:popularity', 'value': 20},
  {'provider_type': 'jw:popular_7_day', 'value': 428},
  {'provider_type': 'imdb:score', 'value': 9.2},
  {'provider_type': 'imdb:multiplied', 'value': 3839776.4},
  {'provider_type': 'tmdb:popularity', 'value': 188.439},
  {'provider_type': 'jw:trending_7_day', 'value': 391.81},
  {'provider_type': 'jw:popular_180_day', 'value': 528.75},
  {'provider_type': 'jw:popular_1_day', 'value': 752.8},
  {'provider_type': 'jw:viral', 'value': 0.12},
  {'provider_type': 'tmdb:score', 'value': 8.6},
  {'provider_type': 'imdb:votes', 'value': 417367},
  {'provider_type': 'jw:popular_30_day', 'value': 394.31},
  {'provider_type': 'tmdb:vote

In [45]:
results2 = just_watch.search_for_item(query='jumanji', release_year_from=1995, release_year_until=1995)
first_result2 = results2['items'][0]


dict_keys(['jw_entity_id', 'id', 'title', 'full_path', 'full_paths', 'poster', 'poster_blur_hash', 'original_release_year', 'object_type', 'offers', 'scoring', 'cinema_release_date'])

[]

In [50]:
if first_result.get('title') == get_from_table('FILM_TITLE', film_id, 'FILM_TITLE'):
    provider_abbreviations = list(set([x['package_short_name'] for x in first_result.get('offers', []) if x['monetization_type'] in ['flatrate', 'free', 'ads']]))

In [24]:
first_result.keys()

dict_keys(['jw_entity_id', 'id', 'title', 'full_path', 'full_paths', 'poster', 'poster_blur_hash', 'original_release_year', 'object_type', 'scoring'])

In [ ]:
if first_result.get('title') == get_from_table('FILM_TITLE', film_id, 'FILM_TITLE'):
    provider_abbreviations = list(set([x['package_short_name'] for x in first_result['offers'] if x['monetization_type'] in ['flatrate', 'free', 'ads']]))
    valid_abbr = [x for x in provider_abbreviations if x in my_streaming_services_abbr]
    if len(valid_abbr) > 0:
        valid_full = [abbr_to_full_dict.get(x) for x in valid_abbr]
        film_streaming_services_df = pd.DataFrame(index=range(len(valid_abbr)))
        film_streaming_services_df['FILM_ID'] = film_id
        film_streaming_services_df['STREAMING_SERVICE_ABBR'] = valid_abbr
        film_streaming_services_df['STREAMING_SERVICE_FULL'] = valid_full
        df_to_table(film_streaming_services_df, 'FILM_STREAMING_SERVICES', replace_append='append', verbose=True)